# Named Entity Recognition with BERT

In [1]:
!pip install transformers wordcloud scikit-learn pandas matplotlib seaborn tqdm

In [2]:
import os
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from sklearn.metrics import classification_report, f1_score
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForTokenClassification, get_scheduler
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from wordcloud import WordCloud


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
df = pd.read_csv("ner_dataset.csv", encoding="latin1").ffill()
df.head()

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,Sentence: 1,of,IN,O
2,Sentence: 1,demonstrators,NNS,O
3,Sentence: 1,have,VBP,O
4,Sentence: 1,marched,VBN,O


from matplotlib import pyplot as plt
import seaborn as sns
_df_0.groupby('Word').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_1.groupby('POS').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
plt.subplots(figsize=(8, 8))
df_2dhist = pd.DataFrame({
    x_label: grp['POS'].value_counts()
    for x_label, grp in _df_2.groupby('Word')
})
sns.heatmap(df_2dhist, cmap='viridis')
plt.xlabel('Word')
_ = plt.ylabel('POS')

In [4]:
print(device)

cpu


In [5]:
df.isnull().sum()

,0
Sentence #,0
Word,0
POS,0
Tag,0


In [6]:
df['Sentence #'] = df['Sentence #'].fillna(method='ffill')
df = df.dropna(subset=['Word'])

df.isnull().sum()

print(f"Total sentences: {df['Sentence #'].nunique()}")
print(f"Total tokens: {len(df)}")


<ipython-input-6-9d11773144c5>:1: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['Sentence #'] = df['Sentence #'].fillna(method='ffill')


Total sentences: 47959
Total tokens: 1048575


In [7]:
sentences = df.groupby("Sentence #")["Word"].apply(list).tolist()
tags = df.groupby("Sentence #")["Tag"].apply(list).tolist()

unique_tags = sorted(set(tag for seq in tags for tag in seq))
label2id = {tag: idx for idx, tag in enumerate(unique_tags)}
id2label = {idx: tag for tag, idx in label2id.items()}

train_texts, val_texts, train_labels, val_labels = train_test_split(sentences, tags, test_size=0.1, random_state=42)

In [8]:
ner_tags = df['Tag'].unique()
print("NER Tags:", ner_tags)

pos_tags = df['POS'].unique()
print("POS Tags:", pos_tags)

print("Number of NER Tags:", len(ner_tags))
print("Number of POS Tags:", len(pos_tags))

NER Tags: ['O' 'B-geo' 'B-gpe' 'B-per' 'I-geo' 'B-org' 'I-org' 'B-tim' 'B-art'
 'I-art' 'I-per' 'I-gpe' 'I-tim' 'B-nat' 'B-eve' 'I-eve' 'I-nat']
POS Tags: ['NNS' 'IN' 'VBP' 'VBN' 'NNP' 'TO' 'VB' 'DT' 'NN' 'CC' 'JJ' '.' 'VBD' 'WP'
 '``' 'CD' 'PRP' 'VBZ' 'POS' 'VBG' 'RB' ',' 'WRB' 'PRP$' 'MD' 'WDT' 'JJR'
 ':' 'JJS' 'WP$' 'RP' 'PDT' 'NNPS' 'EX' 'RBS' 'LRB' 'RRB' '$' 'RBR' ';'
 'UH' 'FW']
Number of NER Tags: 17
Number of POS Tags: 42


In [9]:
print("Label to ID mapping:")
print(label2id)
print("\nID to Label mapping:")
print(id2label)


Label to ID mapping:
{'B-art': 0, 'B-eve': 1, 'B-geo': 2, 'B-gpe': 3, 'B-nat': 4, 'B-org': 5, 'B-per': 6, 'B-tim': 7, 'I-art': 8, 'I-eve': 9, 'I-geo': 10, 'I-gpe': 11, 'I-nat': 12, 'I-org': 13, 'I-per': 14, 'I-tim': 15, 'O': 16}

ID to Label mapping:
{0: 'B-art', 1: 'B-eve', 2: 'B-geo', 3: 'B-gpe', 4: 'B-nat', 5: 'B-org', 6: 'B-per', 7: 'B-tim', 8: 'I-art', 9: 'I-eve', 10: 'I-geo', 11: 'I-gpe', 12: 'I-nat', 13: 'I-org', 14: 'I-per', 15: 'I-tim', 16: 'O'}


In [10]:
class NERDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, label2id, max_len=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.label2id = label2id
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        words = self.texts[idx]
        labels = self.labels[idx]
        encoding = self.tokenizer(words, is_split_into_words=True, return_offsets_mapping=True,
                                  padding="max_length", truncation=True, max_length=self.max_len)
        offset_mapping = encoding.pop("offset_mapping")
        word_ids = encoding.word_ids()
        aligned_labels = np.ones(len(encoding["input_ids"]), dtype=int) * -100
        previous_word_idx = None
        for i, word_idx in enumerate(word_ids):
            if word_idx is None:
                continue
            if word_idx != previous_word_idx:
                aligned_labels[i] = self.label2id.get(labels[word_idx], 0)
            previous_word_idx = word_idx
        item = {key: torch.tensor(val) for key, val in encoding.items()}
        item["labels"] = torch.tensor(aligned_labels)
        return item

In [11]:
from huggingface_hub import login
from google.colab import userdata


from transformers import AutoTokenizer, AutoModelForTokenClassification

model_name = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(
    model_name,
    num_labels=len(unique_tags),
).to(device)


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
train_dataset = NERDataset(train_texts, train_labels, tokenizer, label2id)
val_dataset = NERDataset(val_texts, val_labels, tokenizer, label2id)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)


# BERT BASED

In [14]:
optimizer = AdamW(model.parameters(), lr=3e-5)
scheduler = get_scheduler("linear", optimizer, num_warmup_steps=0, num_training_steps=len(train_loader)*5)

train_losses, val_f1s = [], []

for epoch in range(10):
    model.train()
    total_loss = 0
    for batch in tqdm(train_loader):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
        total_loss += loss.item()
    avg_loss = total_loss / len(train_loader)
    train_losses.append(avg_loss)

    model.eval()
    preds, trues = [], []
    with torch.no_grad():
        for batch in val_loader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            pred = torch.argmax(outputs.logits, dim=-1).cpu().numpy()
            true = batch["labels"].cpu().numpy()
            for p, t in zip(pred, true):
                for pi, ti in zip(p, t):
                    if ti != -100:
                        preds.append(id2label[pi])
                        trues.append(id2label[ti])
    f1 = f1_score(trues, preds, average="weighted")
    val_f1s.append(f1)
    print(f"Epoch {epoch+1}: Train Loss = {avg_loss:.4f}, Validation F1 = {f1:.4f}")

  1%|          | 20/2698 [01:41<3:45:41,  5.06s/it]


KeyboardInterrupt: 

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(train_losses, label="Train Loss")
plt.plot(val_f1s, label="Validation F1")
plt.xlabel("Epoch")
plt.ylabel("Score")
plt.legend()
plt.title("Training Loss and Validation F1")
plt.grid(True)
plt.show()

In [ ]:
# Run inference on our custom sentence
tokens = ["Elon", "Musk", "is", "the", "CEO", "of", "Tesla", "and", "SpaceX", "based", "in", "the", "United", "States"]
inputs = tokenizer(tokens, is_split_into_words=True, return_tensors="pt", truncation=True, padding=True).to(device)
outputs = model(**inputs).logits
predictions = torch.argmax(outputs, dim=2)[0].cpu().numpy()
predicted_labels = [id2label[p] for p in predictions]
print("Predicted NER Tags:")
print(list(zip(tokens, predicted_labels)))


In [ ]:
text = " ".join(df[df['Tag'] != 'O']['Word'].values)
wordcloud = WordCloud(width=1000, height=600, background_color="white", colormap="viridis").generate(text)
plt.figure(figsize=(14, 8))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title("Word Cloud of Named Entities")
plt.show()

In [ ]:

# Installing seqeval
!pip install transformers datasets seqeval


In [ ]:
# Importing libraries
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, AutoModelForTokenClassification, DataCollatorForTokenClassification, TrainingArguments, Trainer
from datasets import DatasetDict, Dataset
!pip install evaluate
import evaluate
import numpy as np
import matplotlib.pyplot as plt

In [ ]:

# Loading and preprocessing the dataset
df = pd.read_csv("ner_dataset.csv", encoding="latin1")
df = df.fillna(method="ffill")

# Creating the grouped sentences and labels
grouped = df.groupby("Sentence #").agg({"Word": list, "Tag": list}).reset_index()
sentences = grouped["Word"].tolist()
labels = grouped["Tag"].tolist()

# Unique labels for the dataset
label_list = sorted(set(tag for seq in labels for tag in seq))
label2id = {l: i for i, l in enumerate(label_list)}
id2label = {i: l for l, i in label2id.items()}

# Encoding labels to IDs
encoded_labels = [[label2id[tag] for tag in seq] for seq in labels]


In [ ]:
df['Sentence #'] = df['Sentence #'].fillna(method='ffill')
sent_lengths = df.groupby("Sentence #")['Word'].count()

plt.figure(figsize=(10, 5))
sns.histplot(sent_lengths, bins=50, kde=True)
plt.title('Sentence Length Distribution')
plt.xlabel('Number of Tokens')
plt.ylabel('Number of Sentences')
plt.tight_layout()
plt.show()

In [ ]:
from collections import Counter

entity_words = df[df['Tag'] != 'O']['Word']
entity_counts = Counter(entity_words)

# Top 20 named entities
most_common_entities = entity_counts.most_common(20)
words, counts = zip(*most_common_entities)

plt.figure(figsize=(12, 6))
sns.barplot(x=list(words), y=list(counts))
plt.title('Top 20 Named Entities')
plt.xticks(rotation=45)
plt.xlabel('Entity Word')
plt.ylabel('Frequency')
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
top_pos = df['POS'].value_counts().head(20)
sns.barplot(x=top_pos.index, y=top_pos.values)
plt.title('Top 20 POS Tags')
plt.xticks(rotation=45)
plt.xlabel('POS Tag')
plt.ylabel('Frequency')
plt.tight_layout()
plt.show()

In [ ]:

# Tokenize inputs
model_name = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, padding="max_length", is_split_into_words=True, max_length=128)
    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[i][word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs


In [ ]:

# Prepare Hugging Face dataset
dataset = Dataset.from_dict({
    "tokens": sentences,
    "ner_tags": encoded_labels
})
dataset = dataset.train_test_split(test_size=0.1)


def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)
    labels = []
    for i, label in enumerate(examples["labels"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        label_ids = []
        previous_word_idx = None
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs


In [ ]:
# Check sample label encoding
print("Sample original tags:", tags[0])
print("Encoded tags:", encoded_labels[0])
print("Back to label names:", [id2label[i] for i in encoded_labels[0]])


In [ ]:

# Load model
model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=len(label_list), id2label=id2label, label2id=label2id)
data_collator = DataCollatorForTokenClassification(tokenizer)


In [ ]:
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split

# Define label mappings from string tags to integers
label_list = sorted(set(tag for seq in tags for tag in seq))
label2id = {l: i for i, l in enumerate(label_list)}
id2label = {i: l for l, i in label2id.items()}

# Encoding string tags to integer IDs
encoded_labels = [[label2id[tag] for tag in seq] for seq in tags]

# Spliting into train and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(
    sentences, encoded_labels, test_size=0.1, random_state=42
)

# Building Hugging Face dataset
dataset = DatasetDict({
    "train": Dataset.from_dict({"tokens": train_texts, "ner_tags": train_labels}),
    "test": Dataset.from_dict({"tokens": val_texts, "ner_tags": val_labels}),
})

# Tokenization and label alignment function
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"],
        truncation=True,
        is_split_into_words=True,
        padding="max_length",
        max_length=128
    )

    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

# Apply the tokenization and label alignment
tokenized_datasets = dataset.map(tokenize_and_align_labels, batched=True)

# Convert to PyTorch DataLoaders
from torch.utils.data import DataLoader

train_dataloader = DataLoader(tokenized_datasets["train"], batch_size=16, shuffle=True)
eval_dataloader = DataLoader(tokenized_datasets["test"], batch_size=16)


In [ ]:
print(id2label)

In [ ]:

def predict_ner_tags(sentence, model, tokenizer, id2label):
    model.eval()
    encoding = tokenizer(sentence,
                         return_tensors="pt",
                         return_offsets_mapping=True,
                         is_split_into_words=False,
                         truncation=True)
    word_ids = encoding.word_ids()
    inputs = {k: v.to(model.device) for k, v in encoding.items() if k in ["input_ids", "attention_mask"]}

    with torch.no_grad():
        outputs = model(**inputs)
    predictions = torch.argmax(outputs.logits, dim=-1)[0].cpu().numpy()
    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
    previous_word_idx = None
    final_tokens = []
    final_labels = []

    for token, pred_label_id, word_idx in zip(tokens, predictions, word_ids):
        if token in tokenizer.all_special_tokens:
            continue
        label = id2label[pred_label_id]
        if word_idx != previous_word_idx:
            final_tokens.append(token)
            final_labels.append(label)
        else:
            final_tokens[-1] += token.replace("##", "")
        previous_word_idx = word_idx

    print("\nNER Prediction:")
    for token, label in zip(final_tokens, final_labels):
        print(f"{token:15s} --> {label}")

# Run it on your test input
test_sentence = "Elon Musk is the CEO of Tesla and SpaceX, based in the United States."
predict_ner_tags(test_sentence, model, tokenizer, id2label)


# distibert-base-cased

In [ ]:
from transformers import DistilBertTokenizerFast

distilbert_model_name = "distilbert-base-cased"
distilbert_tokenizer = DistilBertTokenizerFast.from_pretrained(distilbert_model_name)

#tokenize inpus
def tokenize_and_align_labels_distilbert(examples, label_all_tokens = False):
    tokenized_inputs = distilbert_tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)
    labels = []

    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []

        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

tokenized_distil = dataset.map(tokenize_and_align_labels_distilbert, batched=True)


In [ ]:
from transformers import DistilBertForTokenClassification

distilbert_model = DistilBertForTokenClassification.from_pretrained(
    distilbert_model_name,
    num_labels=len(label2id),
    id2label=id2label,
    label2id=label2id
).to(device)

In [ ]:
from torch.utils.data import DataLoader
from torch.optim import AdamW
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=distilbert_tokenizer)

train_dataloader_distil = DataLoader(train_dataset, batch_size=32, collate_fn=data_collator)
val_dataloader_distil = DataLoader(val_dataset, batch_size=32, collate_fn=data_collator)


train_dataset_distil = tokenized_distil["train"]
val_dataset_distil = tokenized_distil["test"]


# Define optimizer and scheduler
from transformers import get_scheduler

optimizer = AdamW(distilbert_model.parameters(), lr=5e-5)

num_training_steps = len(train_dataloader_distil) * 10
lr_scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)


In [ ]:
#Training for DistilBert
from tqdm.auto import tqdm

train_losses, val_f1s = [], []

for epoch in range(10):
  distilbert_model.train()
  total_loss = 0
  for batch in tqdm(train_dataloader_distil, desc=f"DistilBERT Epoch {epoch+1}"):
      batch = {k: v.to(device) for k, v in batch.items()}
      if 'token_type_ids' in batch:
        del batch['token_type_ids']
      outputs = distilbert_model(**batch)
      loss = outputs.loss
      loss.backward()
      optimizer.step()
      lr_scheduler.step()
      optimizer.zero_grad()
      total_loss += loss.item()
      avg_loss = total_loss / len(train_dataloader_distil)
  train_losses.append(avg_loss)
  distilbert_model.eval()
  distil_preds, distil_trues = [], []
  with torch.no_grad():
      for batch in val_dataloader_distil:
          batch = {k: v.to(device) for k, v in batch.items()}
          if 'token_type_ids' in batch:
            del batch['token_type_ids']
          outputs = distilbert_model(**batch)
          pred = torch.argmax(outputs.logits, dim=-1).cpu().numpy()
          true = batch["labels"].cpu().numpy()
          for p, t in zip(pred, true):
              for pi, ti in zip(p, t):
                  if ti != -100:
                      distil_preds.append(id2label[pi])
                      distil_trues.append(id2label[ti])
  distil_f1 = f1_score(distil_trues, distil_preds, average="weighted")
  val_f1s.append(distil_f1)
  print(f"[DistilBERT] Epoch {epoch+1}: Train Loss = {avg_loss:.4f}, Validation F1 = {distil_f1:.4f}")


In [ ]:
from torch.optim.lr_scheduler import ReduceLROnPlateau
optimizer = AdamW(distilbert_model.parameters(), lr=5e-5)

lr_scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2, verbose=True)
from sklearn.metrics import f1_score, precision_score, recall_score


In [ ]:
train_losses, val_losses, val_f1s, val_precisions, val_recalls = [], [], [], [], []
distil_preds, distil_trues = [], []

for epoch in range(10):
    distilbert_model.train()
    total_loss = 0
    for batch in tqdm(train_dataloader_distil, desc=f"DistilBERT Epoch {epoch+1}"):
        batch = {k: v.to(device) for k, v in batch.items()}
        if 'token_type_ids' in batch:
            del batch['token_type_ids']
        outputs = distilbert_model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        total_loss += loss.item()

    avg_train_loss = total_loss / len(train_dataloader_distil)
    train_losses.append(avg_train_loss)

    distilbert_model.eval()
    distil_preds, distil_trues = [], []
    val_loss = 0

    with torch.no_grad():
        for batch in val_dataloader_distil:
            batch = {k: v.to(device) for k, v in batch.items()}
            if 'token_type_ids' in batch:
                del batch['token_type_ids']

            outputs = distilbert_model(**batch)
            val_loss += outputs.loss.item()

            preds = torch.argmax(outputs.logits, dim=-1).cpu().numpy()
            trues = batch["labels"].cpu().numpy()

            for p, t in zip(preds, trues):
                for pi, ti in zip(p, t):
                    if ti != -100:
                        distil_preds.append(id2label[pi])
                        distil_trues.append(id2label[ti])

    avg_val_loss = val_loss / len(val_dataloader_distil)


    distil_preds = [pred for pred in distil_preds if pred not in ['[CLS]', '[SEP]']]
    distil_trues = [true for true in distil_trues if true not in ['[CLS]', '[SEP]']]


    distil_f1 = f1_score(distil_trues, distil_preds, average="weighted")
    distil_precision = precision_score(distil_trues, distil_preds, average="weighted")
    distil_recall = recall_score(distil_trues, distil_preds, average="weighted")


    val_losses.append(avg_val_loss)
    val_f1s.append(distil_f1)
    val_precisions.append(distil_precision)
    val_recalls.append(distil_recall)

    print(f"[DistilBERT] Epoch {epoch+1}: Train Loss = {avg_train_loss:.4f}, Val Loss = {avg_val_loss:.4f}, "
          f"F1 = {distil_f1:.4f}, Precision = {distil_precision:.4f}, Recall = {distil_recall:.4f}")

    lr_scheduler.step(avg_val_loss)

    for param_group in optimizer.param_groups:
        print(f"Current LR: {param_group['lr']}")


In [ ]:
# Saving model and testing
distilbert_model.save_pretrained("distilbert-bert-ner-model")
tokenizer.save_pretrained("distilbert-bert-ner-model")


In [ ]:
def predict_ner_tags(sentence, model, tokenizer, id2label):
    model.eval()

    # Tokenize the input sentence and get word IDs
    encoding = tokenizer(sentence,
                         return_tensors="pt",
                         return_offsets_mapping=True,
                         is_split_into_words=False,
                         truncation=True)

    word_ids = encoding.word_ids()
    inputs = {k: v.to(model.device) for k, v in encoding.items() if k in ["input_ids", "attention_mask"]}

    # Get model predictions
    with torch.no_grad():
        outputs = model(**inputs)
    predictions = torch.argmax(outputs.logits, dim=-1)[0].cpu().numpy()
    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
    previous_word_idx = None
    final_tokens = []
    final_labels = []

    for token, pred_label_id, word_idx in zip(tokens, predictions, word_ids):
        if token in tokenizer.all_special_tokens:
            continue
        if token in [',', '.', '!', '?', ':', ';']:
            continue
        label = id2label[pred_label_id]
        if label.startswith("I-") and final_labels and final_labels[-1].startswith("O"):
            label = "O"
        if word_idx != previous_word_idx:
            final_tokens.append(token)
            final_labels.append(label)
        else:
            final_tokens[-1] += token[2:]
        previous_word_idx = word_idx

    # Print the results
    print("\nNER Prediction:")
    for token, label in zip(final_tokens, final_labels):
        print(f"{token:15s} --> {label}")

# Example usage
test_sentence = "Elon Musk is the CEO of Tesla and SpaceX, based in the United States."
predict_ner_tags(test_sentence, distilbert_model, distilbert_tokenizer, id2label)


test_sentence = "Mr. Mohan Das Karamchand Gandhi was the person behind betrayal of electing Prime Minister of India"
predict_ner_tags(test_sentence, distilbert_model, distilbert_tokenizer, id2label)


# Model 3